In [25]:
!pip install langchain-ollama

import os, getpass

from langchain_core.callbacks import CallbackManagerForLLMRun, AsyncCallbackManagerForLLMRun
from langchain_core.outputs import ChatResult


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

os.environ['OPENAI_API_KEY'] = os.environ.get('OPEN_AI_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
ollama_chat = ChatOllama(model="deepseek-r1:14b", temperature=0)

In [27]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Say Hello", name="Adam")

# Create a message list
messages = [msg]

#Invoke a model with a list of messages
gpt4o_chat.invoke(messages)
# ollama_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-6efb72d5-5532-41c3-a22c-6e6e35276d37-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})